In [ ]:
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import json

spark = SparkSession.builder.appName("Desafio_XPTO").getOrCreate()

In [ ]:
df = spark.read.json(c's3://raw'+'*.json')

In [ ]:
df = df.withColumn("tweet_text", decode(encode("tweet_text", 'ISO-8859-1'), 'UTF-8'))

In [ ]:
emocaopos = [":D",":)",":]",":-)",":P",":p",":v",": )"]
emocaoneg = [":(",":[",":{"]

todas_emocoes = [':D', ':)', ':]', ':-)', ':P', ':p', ':v', ': )',':(', ':[', ':{']

In [ ]:
df_emocao = df.withColumn(":D", locate(":D", col("tweet_text")))\
                .withColumn(":)", locate(":)", col("tweet_text")))\
                    .withColumn(":]", locate(":]", col("tweet_text")))\
                        .withColumn(":-)", locate(":-)", col("tweet_text")))\
                            .withColumn(":P", locate(":P", col("tweet_text")))\
                                .withColumn(":p", locate(":p", col("tweet_text")))\
                                    .withColumn(":v", locate(":v", col("tweet_text")))\
                                        .withColumn(": )", locate(": )", col("tweet_text")))\
                                            .withColumn(":(", locate(":(", col("tweet_text")))\
                                                .withColumn(":[", locate(":[", col("tweet_text")))\
                                                    .withColumn(":{", locate(":{", col("tweet_text")))

df_emocao.show()

In [ ]:
df_simbolos = df_emocao.withColumn('simbolo', least(*[(when(col(c).isNull() | (col(c) == 0), "None").otherwise(c))for c in df_emocao.columns[3:]]))

df_simbolos = df_simbolos.drop(*todas_emocoes)

df_completa = df_simbolos.withColumn('Sentimento', when(df_simbolos.simbolo.isin(emocaopos), 'positivo')\
                                                .when(df_simbolos.simbolo.isin(emocaoneg), 'negativo')\
                                                    .otherwise("neutra"))

df_completa.show(truncate=True)


In [ ]:
df_out = df_completa.withColumn('Ano', date_format(df_completa.tweet_date, "yyyy"))\
                        .withColumn('Mes', date_format(df_completa.tweet_date, "MM"))\
                            .withColumn('Dia', date_format(df_completa.tweet_date, "dd"))
                            
df_out.show()

df_out.write.partitionBy("Ano","Mes","Dia").parquet("s3://ref", mode = 'overwrite')
print("Done!!")